In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_unixtime, hour, when
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

In [4]:
# Load the data
spark = SparkSession.builder.appName("TelecomActivityPreprocessing").getOrCreate()
data = spark.read.text("/user1/dataverse_files/*.txt")

In [5]:
data.show()

+--------------------+
|               value|
+--------------------+
|1\t1383260400000\...|
|1\t1383260400000\...|
|1\t1383261000000\...|
|1\t1383261000000\...|
|1\t1383261000000\...|
|1\t1383261600000\...|
|1\t1383261600000\...|
|1\t1383262200000\...|
|1\t1383262200000\...|
|1\t1383262800000\...|
|1\t1383262800000\...|
|1\t1383263400000\...|
|1\t1383263400000\...|
|1\t1383264000000\...|
|1\t1383264000000\...|
|1\t1383264600000\...|
|1\t1383264600000\...|
|1\t1383265200000\...|
|1\t1383265800000\...|
|1\t1383265800000\...|
+--------------------+
only showing top 20 rows



In [6]:
# Preprocess the data
preprocessed_data = (
    data.selectExpr("split(value, ' ') as values")
    .select(
        col("values").getItem(0).cast("double").alias("Square_id"),
        (col("values").getItem(1).cast("bigint") / 1000).alias("Timestamp"),#.cast("timestamp").alias("Timestamp"),
        col("values").getItem(2).cast("double").alias("SMS_in_activity"),
        col("values").getItem(3).cast("double").alias("SMS_out_activity"),
        col("values").getItem(4).cast("double").alias("Call_in_activity"),
        col("values").getItem(5).cast("double").alias("Call_out_activity"),
        col("values").getItem(6).cast("double").alias("Internet_traffic_activity"),
        col("values").getItem(7).cast("int").alias("Country_code")
    )
    .filter(
        col("Square_id").isNotNull()
        & col("Timestamp").isNotNull()
        & col("SMS_in_activity").isNotNull()
        & col("SMS_out_activity").isNotNull()
        & col("Call_in_activity").isNotNull()
        & col("Call_out_activity").isNotNull()
        & col("Internet_traffic_activity").isNotNull()
        & col("Country_code").isNotNull()
    ) # Feature engineering
    .withColumn(
        "Hour_of_day",
        hour(from_unixtime(col("Timestamp")))
    )
    .withColumn(
        "Moment_of_day",
        when((col("Hour_of_day") >= 6) & (col("Hour_of_day") < 12), "Morning")
        .when((col("Hour_of_day") >= 12) & (col("Hour_of_day") < 18), "Afternoon")
        .when((col("Hour_of_day") >= 18) & (col("Hour_of_day") < 24), "Evening")
        .otherwise("Night")
    )
    .withColumn(
        "Weekday_or_weekend",
        when(col("Timestamp").isin(1, 7), "Weekend")
        .otherwise("Weekday")
    )
)

In [7]:
preprocessed_data.printSchema()

root
 |-- Square_id: double (nullable = true)
 |-- Timestamp: double (nullable = true)
 |-- SMS_in_activity: double (nullable = true)
 |-- SMS_out_activity: double (nullable = true)
 |-- Call_in_activity: double (nullable = true)
 |-- Call_out_activity: double (nullable = true)
 |-- Internet_traffic_activity: double (nullable = true)
 |-- Country_code: integer (nullable = true)
 |-- Hour_of_day: integer (nullable = true)
 |-- Moment_of_day: string (nullable = false)
 |-- Weekday_or_weekend: string (nullable = false)



In [8]:
# let's reduce the dataframe size summarizing SMS and call data
from pyspark.sql import functions as F

preprocessed_data = preprocessed_data.withColumn('SMS_activity', F.col('SMS_in_activity') + F.col('SMS_out_activity'))
preprocessed_data = preprocessed_data.withColumn('Call_activity', F.col('Call_in_activity') + F.col('Call_out_activity'))

In [10]:
preprocessed_data = preprocessed_data.drop('SMS_in_activity', 'SMS_out_activity', 'Call_in_activity', 'Call_out_activity', 'Hour_of_day')

In [13]:
preprocessed_data.printSchema()

root
 |-- Square_id: double (nullable = true)
 |-- Timestamp: double (nullable = true)
 |-- Internet_traffic_activity: double (nullable = true)
 |-- Country_code: integer (nullable = true)
 |-- Moment_of_day: string (nullable = false)
 |-- Weekday_or_weekend: string (nullable = false)
 |-- SMS_activity: double (nullable = true)
 |-- Call_activity: double (nullable = true)



In [ ]:
#preparing for modeling (cat values, scaling, splitting)
# handle categorical values
from pyspark.ml.feature import OneHotEncoder, StringIndexer

# Create StringIndexer for categorical columns
indexer1 = StringIndexer(inputCol='Moment_of_day', outputCol='Moment_of_day_index')
indexer2 = StringIndexer(inputCol='Weekday_or_weekend', outputCol='Weekday_or_weekend_index')

# Apply StringIndexer to get numerical indices
indexed_df = indexer1.fit(preprocessed_data).transform(preprocessed_data)
indexed_df = indexer2.fit(indexed_df).transform(indexed_df)

# Apply OneHotEncoder to convert indices into binary vectors
encoder = OneHotEncoder(inputCols=['Moment_of_day_index', 'Weekday_or_weekend_index'],
                        outputCols=['Moment_of_day_encoded', 'Weekday_or_weekend_encoded'])
encoded_model = encoder.fit(indexed_df)
encoded_df = encoded_model.transform(indexed_df)

In [ ]:
# scaling numerical features
from pyspark.ml.feature import StandardScaler

# Create StandardScaler for numerical columns
scaler = StandardScaler(inputCols=['Square_id', 'Timestamp', 'Internet_traffic_activity'],
                        outputCols=['Scaled_Square_id', 'Scaled_Timestamp', 'Scaled_Internet_traffic_activity'])

# Fit the scaler on the data and transform the features
scaled_df = scaler.fit(encoded_df).transform(encoded_df)


In [17]:
# Prepare the data for training the autoencoder
X_train = preprocessed_data.select("*")  # "*" to take all input features

In [20]:
# Build the autoencoder model
#input_dim = X_train.shape[1]
input_dim = len(X_train.columns)
encoding_dim = 32

autoencoder = Sequential([
    Dense(encoding_dim, activation='relu', input_shape=(input_dim,)),
    Dense(input_dim, activation='sigmoid')
])

2023-09-20 17:35:21.548387: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-09-20 17:35:21.557477: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-09-20 17:35:21.571976: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (muhammad-VM): /proc/driver/nvidia/version does not exist
2023-09-20 17:35:21.771591: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# Compile and train the autoencoder model
#autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# autoencoder.fit(X_train, X_train, epochs=10, batch_size=32) 
# ^ this won't work as we can't train a Keras model on a SparkDataframe 
# and data is too big to be converted to a Pandas Dataframe.

Since we cannot directly fit a Keras model on a Spark DataFrame, we need to convert it to an RDD (Resilient Distributed Dataset). We use the rdd property of the Spark DataFrame to achieve this. <br><br>

We use the foreachPartition method of the RDD to train the autoencoder model on each partition of the data. This allows to distribute the training process across multiple workers in the Spark cluster.

In [22]:
rdd = X_train.rdd

Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [25]:
# Define a function to train the autoencoder model on each partition
def train_autoencoder(partition):

    # Create a new autoencoder model for each partition
    autoencoder = Sequential([
        Dense(encoding_dim, activation='relu', input_shape=(input_dim,)),
        Dense(input_dim, activation='sigmoid')
    ])

    # Compile and fit the autoencoder model on the partition data
    autoencoder.compile(optimizer='adam', loss='mse')
    autoencoder.fit(partition, partition, epochs=10, batch_size=32)

# Train the autoencoder model on each partition of the RDD
rdd.foreachPartition(train_autoencoder)

# Detect anomalies using the trained autoencoder
reconstructed_data = autoencoder.predict(X_train)
mse = tf.keras.losses.mean_squared_error(X_train, reconstructed_data)
anomaly_threshold = mse.mean() + mse.std() * 2  # Adjust the threshold as per your requirements

anomalies = X_train[mse > anomaly_threshold]


2023-09-20 17:59:57.966268: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-09-20 17:59:57.966340: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-09-20 17:59:59.968200: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-09-20 17:59:59.968379: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-09-20 18:00:13.052426: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 17.0 failed 1 times, most recent failure: Lost task 0.0 in stage 17.0 (TID 289) (10.0.2.15 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 609, in process
    out_iter = func(split_index, iterator)
  File "/usr/local/spark/python/pyspark/rdd.py", line -1, in pipeline_func
  File "/usr/local/spark/python/pyspark/rdd.py", line -1, in pipeline_func
  File "/usr/local/spark/python/pyspark/rdd.py", line -1, in pipeline_func
  File "/usr/local/spark/python/pyspark/rdd.py", line 614, in func
    rand.shuffle(samples)
  File "/usr/local/spark/python/pyspark/rdd.py", line 934, in func
    t

2023-09-20 18:00:16.156045: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-09-20 18:00:16.156249: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-09-20 18:00:16.156330: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (muhammad-VM): /proc/driver/nvidia/version does not exist
2023-09-20 18:00:16.157712: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-20 18:00:16,504 WARN scheduler.TaskSetManager: Lost task 1.0 in stage 17.0 (TID 290) (10.0.2.15 executor dr

2023-09-20 21:39:28,585 ERROR netty.Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:113)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:112)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:548)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:547)
	at org

2023-09-20 21:39:38,587 ERROR netty.Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:113)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:112)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:548)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:547)
	at org

...

2023-09-20 21:48:28,584 ERROR netty.Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:113)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:112)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:548)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:547)
	at org

2023-09-20 21:48:38,591 ERROR netty.Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:113)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:112)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:548)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:547)
	at org

2023-09-20 21:48:48,585 ERROR netty.Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:113)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:112)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:548)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:547)
	at org

2023-09-20 21:48:58,585 ERROR netty.Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:113)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:112)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:548)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:547)
	at org

2023-09-20 21:49:08,591 ERROR netty.Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:113)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:112)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:548)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:547)
	at org

To measure the performance of the autoencoder model, we can use the reconstruction error as a metric. The reconstruction error is typically measured as the mean squared error (MSE) between the original input and its reconstruction. A lower reconstruction error indicates better performance.

To evaluate the model’s performance, we can calculate the MSE between the original input data X_train and its corresponding reconstructed data reconstructed_data

In [ ]:
mse = tf.keras.losses.mean_squared_error(X_train, reconstructed_data)

In [ ]:
# Plotting anomalies versus normal items
plt.scatter(X_train['Timestamp'], X_train['SMS_in_activity'], label='Normal')
plt.scatter(anomalies['Timestamp'], anomalies['SMS_in_activity'], color='red', label='Anomaly')
plt.xlabel('Timestamp')
plt.ylabel('SMS-in Activity')
plt.title('Anomalies Detected')
plt.legend()
plt.show()

In [ ]:
# Stop the Spark session
spark.stop()

In [2]:
# =============================================

In [3]:
# tuning model

In [ ]:
# Build multiple autoencoder models with different initializations or hyperparameters
num_models = 5
autoencoders = []
for _ in range(num_models):
    autoencoder = Sequential([
        Dense(encoding_dim, activation='relu', input_shape=(input_dim,)),
        Dense(input_dim, activation='sigmoid')
    ])
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    autoencoders.append(autoencoder)

# Train each autoencoder model separately
for autoencoder in autoencoders:
    autoencoder.fit(X_train, X_train, epochs=10, batch_size=32)

# Combine the outputs of all autoencoder models using ensemble methods (e.g., averaging)
reconstructed_data = np.mean([autoencoder.predict(X_train) for autoencoder in autoencoders], axis=0)

# Calculate the mean squared error (MSE) between the original input and ensemble reconstruction
mse = tf.keras.losses.mean_squared_error(X_train, reconstructed_data)

# Set a threshold based on your specific requirements to classify data points as anomalies or normal
anomaly_threshold = mse.mean() + mse.std() * 2

# Identify anomalies based on the MSE threshold
anomalies = X_train[mse > anomaly_threshold]